# **1) Imagínese trabajando en la actualización de la estimación de las probabilidades de default (PD) de todas las carteras crediticias de Santander Argentina y se cuenta con toda la información histórica para poder calcularlas.**
- Desafío a: ¿Puede plantear el cálculo de las Frecuencias de Default Observadas (FDO u ODR)?
- Desafío b: Se le ha solicitado al equipo de Metodología “analizar la segmentación desde cero” para verificar que la utilizada actualmente sea óptima y no se esté perdiendo una oportunidad de maximizar la precisión de las estimaciones. ¿Cómo plantearía este análisis de segmentación?


**Desafío a: Cálculo de las Frecuencias de Default Observadas (FDO/ODR)**
El cálculo de las freciencias de default observadas, es una medida de riesgo útil para entender el comportamiento de una cartera en base a su comportamiento histórico.

Se puede plantear de la siguiente manera:
FDO = Cantidad de clientes que entraron en default dentro de un período / Cantidad de clientes al inicio del período observado

Una forma de construir el ratio, es partiendo de una base con todos los clientes activos al principio de un período determinado (por ejemplo todos los créditos otorgados/vigentes al 1/1/2025). Luego, se registra un seguimiento de cada cliente durante una ventana temporal definida, por ejemplo de 12 meses, a modo de evaluar el comportamiento de los mismos. Finalizado el período, se contabiliza la cantidad de operaciones caídas en default, y se dividen sobre el total de los vigentes al inicio del período.
Como resultado, se obriene la FDO de esta cohorte, que suele ser expuesta en un formato de vintage, a modo de comparar la performance y evolución de distintas originaciones o segmentos. La misma también es útil en el proceso de parametrización de PDs.

**Desafío b: Análisis de segmentación**
A modo de validar la segmentación vigente, partiría en primer lugar del testeo de la misma aplicando la métrica KS (Kolmogorov-Smirnov).

Esta, permite çomparar la máxima diferencia entre la distribución acumulada de clientes buenos y malos (aquellos que no entraron y sí entraron default) generada por el modelo de segmentación bajo análisis.
Un KS elevado, indica que la segmentación es eficiente para discriminar el riesgo, lo cual implica una diferencia clara entre ambos grupos lograda por la segmentación. En cambio, un indicador de KS bajo indicaría que la segmentación no logra captar una diferenciación representativa.

En caso de que el testeo de la eficiencia de la segmentación no resulte favorable, probaría con nuevas variables como: nivel de actividad del cliente, antiguedad, edad, etc. A partir de ellas, volvería a ejecutar la validación a través del KS para definir y justificar la implementación de una nueva segmentación asegurando la calidad de la misma.


# **2) Imagínese trabajando en la actualización de la estimación de la Loss Given Default (LGD) de todas las carteras crediticias de Santander Argentina y se cuenta con toda la información histórica para poder calcularlas.**
Se desea conocer la evolución histórica de los recobros respecto a las deudas iniciales para cada fecha inicial de default.
Desafío: ¿Cómo armaría un triángulo de desarrollo de la LGD en el que se pueda observar el % recobrado en cada mes del ciclo recuperatorio? ¿Habría que tener consideración de la proyección de los ciclos abiertos en este análisis?


A modo de generar un triángulo de desarrollo de la LGD, aplicaría la metología  Chain-Ladder. La misma resulta útil para analizar la evolución del recupero de a lo largo del tiempo, permitiendo observar patrones y estimar comportamientos futuros.

En la práctica lo implementaría de la siguiente manera:
- A partir de la base histórica de operaciones en default, organizaría los datos en cohortes según la fecha de entrada en mora
- Agruparía los montos recuperados a partir del histórico de cobranzas, en función del tiempo transcurrido desde el default
- De esta forma, se generaría una tabla en formato de triángulo, donde cada celda representa el monto total recuperado una vez transcurridos N meses desde el default
- Finalmente, calcularía el % acumulado de recobros respecto al saldo inicial en mora.

En cuanto al tratamiento de ciclos abiertos (aquellas cohortes recientes que todavía no alcanzaron todo el horizonte de recupero definido), consideraría proyectar los meses faltantes en función de los promedios observados en cohortes anteriores completas, a modo de estimar una LGD para las mismas.

# **3) Imagínese trabajando en la actualización de la estimación de los factores de conversión del crédito (CCF) de cuentas corrientes.**
Se le ha solicitado a Metodología revisar la segmentación del CCF para poder tener un impacto razonable de una iniciativa de aumento masivo de los límites.
Desafío: ¿Qué propuestas evaluaría para poder modificar la segmentación acorde a esta nueva gestión?

Ante la necesidad de evaluar un posible aumento masivo de límites, sería adecuado comenzar con una revisión de la segmentación actual del parámetro CCF, para poder asegurar la correcta aplicación del mismo.

Para ello, haría lo siguiente
- Analizar el uso actual de límites: mediría el nivel actual de utilización del límite para todos los clientes para identificar cuántos muestran un nivel ajustado cercano al 100% de la línea
- Ejecutar pruebas al actual esquema de segmentación, y de ser necesario probar nuevas alternativas de agupamiento de clientes como: qué % del límite utilizan habitualmente, la variación mes a mes del uso, los antecedentes de mora.
- Simular el impacto de los nuevos límites
- Comparar distintos tipos de segmentación a modo de seleccionar aquella que refleje bien las diferencias de comportamiento en el uso de líneas

## **4) Escriba una consulta para imprimir periodo, sucursal y suma del saldo , solamente para los productos vigentes cuando el saldo total de la sucursal sea mayor a $10.000 y el periodo Marzo 2019 ordenados del menor saldo al mayor.**

In [ ]:
!pip install duckdb

In [ ]:
import pandas as pd
import sqlite3

# Se incorporan las tablas
cuentas_data = {
    'PERIODO': [201904, 201904, 201904, 201903, 201903, 201903, 201903, 201903],
    'NUP': [671, 672, 673, 674, 675, 676, 677, 678],
    'SUCURSAL': [268, 708, 38, 708, 708, 234, 212, 123],
    'CUENTA': [1964, 2321, 2132, 1231, 3454, 6456, 7655, 7866],
    'PRODUCTO': [42, 40, 41, 40, 42, 46, 42, 41],
    'SALDO': [5603, 5808, 5755, 30493, 4760, 8730, 68747, 16287]
}
cuentas = pd.DataFrame(cuentas_data)

productos_data = {
    'PERIODO': [201903, 201903, 201903, 201903, 201903, 201904, 201904, 201904],
    'PRODUCTO': [42, 43, 44, 45, 46, 42, 44, 43],
    'ESTADO': ['VIGENTE', 'VIGENTE', 'VIGENTE', 'VIGENTE', 'BAJA', 'VIGENTE', 'BAJA', 'VIGENTE']
}
productos = pd.DataFrame(productos_data)

# Genera y almacena la BBDD
conn = sqlite3.connect(':memory:')

# Se vinculan las tablas a la BBDD
cuentas.to_sql('cuentas', conn, index=False, if_exists='replace')
productos.to_sql('productos', conn, index=False, if_exists='replace')

,PERIODO,SUCURSAL,SALDO_TOTAL
0,201903,212,68747


In [ ]:
# Desarrollo de la consulta
query = """
    SELECT
        c.PERIODO,
        c.SUCURSAL,
        SUM(c.SALDO) AS SALDO_TOTAL
    FROM cuentas c
    JOIN productos p
        ON c.PRODUCTO = p.PRODUCTO
        AND c.PERIODO = p.PERIODO
    WHERE
        c.PERIODO = 201903
        AND p.ESTADO = 'VIGENTE'
    GROUP BY
        c.PERIODO, c.SUCURSAL
    HAVING
        SUM(c.SALDO) > 10000
    ORDER BY
        SALDO_TOTAL ASC
"""

# Se ejecuta la consulta y se imprime el resultado
resultado = pd.read_sql_query(query, conn)
resultado


,PERIODO,SUCURSAL,SALDO_TOTAL
0,201903,212,68747


**Conclusión**
Del análisis realizado, se observa que para el período evaluado, únicamente la sucursal 212 cumple con las condiciones establecidas: contar con un saldo total superior a $10.000 asociado a productos con estado vigente.

Por prudencia, se excluyeron de la consulta aquellos productos cuyo estado no se encuentra especificado en la tabla de referencia, a modo de evitar asumir un estado incorrecto para estos. En un caso real, sería bueno consultar con las áreas de negocio para definir el tratamiento adecuado de estos registros faltantes.

#**5) Imagine la situación en que se están entrevistando a muchos candidatos de diferentes universidades mediante anuncios y desafíos de codificación.**
Escriba una consulta para imprimir anuncio_id, candidato_id, nombre y las sumas de total de intentos de desafíos, total  desafíos aceptados, total vistas y total vistas únicas para cada anuncio ordenado por anuncio_id. Excluye el anuncio del resultado si las cuatro sumas son 0.


In [ ]:
import pandas as pd
import sqlite3

#Se incorporan las tablas
anuncios = pd.DataFrame({
    'anuncio_id': [66406, 66556, 94828],
    'candidato_id': [17973, 79153, 80275],
    'nombre': ['Rosa', 'Angela', 'Franco']
})

universidades = pd.DataFrame({
    'universidad_id': [11219, 32473, 56685],
    'anuncio_id': [66406, 66556, 94828]
})

desafios = pd.DataFrame({
    'desafio_id': [18765, 47127, 60292, 72974],
    'universidad_id': [11219, 11219, 32473, 56685]
})

vistas = pd.DataFrame({
    'desafio_id': [47127, 47127, 18765, 18765, 75516, 60292, 72974, 75516],
    'total_vistas': [26, 15, 43, 72, 35, 11, 41, 75],
    'total_vistas_unicas': [19, 14, 10, 13, 17, 10, 15, 11]
})

resultados = pd.DataFrame({
    'desafio_id': [75516, 47127, 47127, 75516, 75516, 72974, 72974, 47127],
    'total_intentos': [34, 27, 56, 74, 83, 68, 82, 28],
    'total_aceptados': [12, 10, 18, 12, 8, 24, 14, 11]
})


conn = sqlite3.connect(':memory:')
anuncios.to_sql('Anuncios', conn, index=False, if_exists='replace')
universidades.to_sql('Universidades', conn, index=False, if_exists='replace')
desafios.to_sql('Desafios', conn, index=False, if_exists='replace')
vistas.to_sql('Vistas', conn, index=False, if_exists='replace')
resultados.to_sql('Resultados', conn, index=False, if_exists='replace')


8

In [ ]:
query = '''
WITH sum_vistas AS (
    SELECT
        desafio_id,
        SUM(total_vistas) AS total_vistas,
        SUM(total_vistas_unicas) AS total_vistas_unicas
    FROM Vistas
    GROUP BY desafio_id
),
sum_resultados AS (
    SELECT
        desafio_id,
        SUM(total_intentos) AS total_intentos,
        SUM(total_aceptados) AS total_aceptados
    FROM Resultados
    GROUP BY desafio_id
)
SELECT
    a.anuncio_id,
    a.candidato_id,
    a.nombre,
    COALESCE(SUM(sr.total_intentos), 0) AS total_intentos,         #Se utiliza COALESCE para evitar valores nulos en la suma de los valores
    COALESCE(SUM(sr.total_aceptados), 0) AS total_aceptados,
    COALESCE(SUM(sv.total_vistas), 0) AS total_vistas,
    COALESCE(SUM(sv.total_vistas_unicas), 0) AS total_vistas_unicas
FROM Anuncios a
LEFT JOIN Universidades u ON a.anuncio_id = u.anuncio_id           #Se anexan las consultas manteniendo los valores de la tabla de la izquierda
LEFT JOIN Desafios d ON u.universidad_id = d.universidad_id
LEFT JOIN sum_resultados sr ON d.desafio_id = sr.desafio_id
LEFT JOIN sum_vistas sv ON d.desafio_id = sv.desafio_id
GROUP BY a.anuncio_id, a.candidato_id, a.nombre                    #Se agrupan por anuncio_id, candidato_id y nombre
HAVING                                                             #Se aplican filtros por las condiciones especificadas
    total_intentos > 0 OR
    total_aceptados > 0 OR
    total_vistas > 0 OR
    total_vistas_unicas > 0
ORDER BY a.anuncio_id
'''

resultado_sql = pd.read_sql_query(query, conn)
resultado_sql


,anuncio_id,candidato_id,nombre,total_intentos,total_aceptados,total_vistas,total_vistas_unicas
0,66406,17973,Rosa,111,39,156,56
1,66556,79153,Angela,0,0,11,10
2,94828,80275,Franco,150,38,41,15


**Conclusión**
En base a la integración de las tablas (anuncios, universidades, desafíos, vistas y resultados), se identificó para cada anuncio y candidato el total de desafíos intentados, aceptados, vistas y vistas únicas.
En este caso, la candidata Rosa, fue la que obtuvo mayor participación, mientras que Angela solo registró vistas sin intentos.



# 6) **Estamos realizamos un concurso de Coding Challenge donde realizamos múltiples exámenes durante 15 días, La fecha de inicio del concurso fue el 01 de marzo de 2022 y la fecha de fiscalización fue el 15 de marzo de 2022.**
Escriba una consulta para imprimir el número total de participantes únicos que realizaron al menos 1 un examen en cada día (a partir del primer día del concurso), y encuentre el legajo y el nombre del participante que realizó el número máximo de exámenes cada día. Si más de un participante tiene el mismo número máximo de envíos, imprima el legajo más bajo. La consulta debe imprimir esta información para cada día del concurso, ordenados por fecha.
Nota: Tener en cuenta que pueden existir exámenes realizados en fechas distintas al concurso.

In [ ]:
import pandas as pd
import sqlite3

#Se incorporan las tablas

participantes = pd.DataFrame({
    'LEGAJO': [304561, 300145, 236454, 25456, 12546, 233011, 156589, 315658],
    'NOMBRE': ['Javi', 'Agus', 'Aye', 'Marco', 'Gabo', 'Lucas', 'Pau', 'Gonza']
})

examenes = pd.DataFrame({
    'FECHA_EXAMEN': [
        '2022-03-05', '2022-03-03', '2022-03-02', '2022-03-02', '2022-03-02',
        '2022-03-04', '2022-03-03', '2022-03-01', '2022-03-03', '2022-03-04',
        '2022-03-06', '2022-03-05', '2022-03-04', '2022-03-01', '2022-03-03',
        '2022-03-04', '2022-03-02', '2022-03-01', '2022-03-05', '2022-03-04',
        '2022-03-03', '2022-03-04'
    ],
    'ID_EXAMEN': [
        9792, 8018, 7509, 6089, 6071,
        8954, 7527, 8265, 6935, 8779,
        7389, 9428, 5138, 8499, 9470,
        6542, 6379, 7581, 6593, 7685,
        8962, 6909
    ],
    'LEGAJO': [
        233011, 300145, 315658, 156589, 233011,
        236454, 12546, 233011, 6935, 25456,
        156589, 156589, 315658, 300145, 12546,
        233011, 25456, 315658, 300145, 236454,
        304561, 304561
    ],
    'PUNTACION': [
        43, 25, 16, 33, 25,
        83, 2, 73, 82, 83,
        19, 94, 2, 21, 57,
        40, 17, 23, 49, 46,
        22, 25
    ]
})

# Definición de tipo de campo a FECHA_EXAMEN
examenes['FECHA_EXAMEN'] = pd.to_datetime(examenes['FECHA_EXAMEN'])

# Se crea y almacena la BBDD en memoria y se vinculan las tablas
conn = sqlite3.connect(':memory:')
participantes.to_sql('Participantes', conn, index=False, if_exists='replace')
examenes.to_sql('Examenes', conn, index=False, if_exists='replace')

22

In [ ]:
# Se define el intervalo de consulta (las fechas del concurso)
inicio = datetime.date(2022, 3, 1)
fin = datetime.date(2022, 3, 6)
fechas_concurso = pd.date_range(inicio,fin).strftime('%Y-%m-%d').tolist()

In [ ]:
# Se genera y almacena una tabla con el rango de fechas del concurso
fechas_df = pd.DataFrame({'FECHA': fechas_concurso})
fechas_df.to_sql('FechasConcurso', conn, index=False, if_exists='replace')

6

In [ ]:
# Desarrollo de la Consulta
query = '''
WITH examenes_por_dia AS (                                               #Se calcula la cantidad de examenes realizados por día por participante
    SELECT
        date(FECHA_EXAMEN) AS FECHA,
        LEGAJO,
        COUNT(*) AS examenes
    FROM Examenes
    WHERE date(FECHA_EXAMEN) BETWEEN '2022-03-01' AND '2022-03-15'
    GROUP BY FECHA, LEGAJO
),
top_por_dia AS (                                                        #Se asigna para cada día, un ranking al aprticipante según la cantidad de exámenes tomados
        FECHA,
        LEGAJO,
        examenes,
        RANK() OVER (PARTITION BY FECHA ORDER BY examenes DESC, LEGAJO ASC) AS rnk
    FROM examenes_por_dia
),
max_por_dia AS (                                                        #Se seleccionan los participantes que presentaron la máxima cantidad de exámenes por día
    SELECT
        FECHA,
        LEGAJO,
        examenes
    FROM top_por_dia
    WHERE rnk = 1
)
SELECT
    e.FECHA,
    COUNT(DISTINCT ep.LEGAJO) AS participantes_unicos,                   #Se calcula la cantidad de participantes únicos por día
    m.LEGAJO,
    p.NOMBRE
FROM
    (SELECT DISTINCT date(FECHA_EXAMEN) AS FECHA FROM Examenes WHERE date(FECHA_EXAMEN) BETWEEN '2022-03-01' AND '2022-03-15') e     #Se genera una lista de las fechas donde hubieron exámenes dentro del rango
LEFT JOIN examenes_por_dia ep ON e.FECHA = ep.FECHA                                             #Se anexan las consultas para obtener el dato de cuántos exámenes se realizaron por día por participante
LEFT JOIN max_por_dia m ON e.FECHA = m.FECHA
LEFT JOIN Participantes p ON m.LEGAJO = p.LEGAJO
GROUP BY e.FECHA, m.LEGAJO, p.NOMBRE
ORDER BY e.FECHA
'''

resultado = pd.read_sql_query(query, conn)
resultado

,FECHA,participantes_unicos,LEGAJO,NOMBRE
0,2022-03-01,3,233011,Lucas
1,2022-03-02,4,25456,Marco
2,2022-03-03,4,12546,Gabo
3,2022-03-04,5,236454,Aye
4,2022-03-05,3,156589,Pau
5,2022-03-06,1,156589,Pau


**Conclusión**
Se muestran las personas que presentaron el puntaje más alto de cada día en que se registraron exámenes

A partir de las tablas, se identificó para cada día del concurso cuántos participantes distintos rindieron al menos un examen, y también quién fue el que más exámenes realizó en ese día (priorizando el legajo más bajo en caso de registrar participantes con el mismo puntaje)

Los resultados también permiten observar el el nivel de participación (partipantes_unicos). El mismo, varió a lo largo del concurso mostrando jornadas con hasta 5 participantes


# **7) Cree un programa que lea las tablas Clientes y Scores, y calcule la PD de cada cliente según el score.**
Luego muestre dos listas con los nombres, saldos, score y pd de los clientes con Peor Score y Mayor Deuda y Mejor Score y Mayor Deuda.
Tener en cuenta:
- La tabla Score puede tener duplicados y deberá traer el de mayor score
- La tabla clientes puede tener duplicados y deberá sumar la deuda.
- El formato de la lista de salida deberá ser todo en String (
  cada uno de los 4 campos)


In [ ]:
import pandas as pd

# Se incorporan las tablas descriptas
#Parámetros
parametros = pd.DataFrame({
    'umbral_desde': [1, 10, 30, 60],
    'umbral_hasta': [10, 30, 60, 100],
    'PD': [0.8, 0.5, 0.3, 0.1],
    'Periodo': [202401, 202401, 202401, 202401]
})

#Clientes
clientes = pd.DataFrame({
    'Id_cliente': [1000, 8000, 9000],
    'Nombre': ['Pepe', 'Juan', 'Roxana'],
    'Apellido': ['Mendez', 'Perez', 'Vazquez'],
    'Deuda': [10000, 50241, 8050]
})

#Scores
scores = pd.DataFrame({
    'Id_cliente': [1000, 8000, 8000, 9000],
    'Score': [50, 20, 30, 10]
})


In [ ]:
# Se limpian los duplicados manteniendo el mayor score por cliente y se suman las deudas por cliente (en caso que hubieran registros duplicados)
scores_max = scores.groupby('Id_cliente', as_index=False)['Score'].max()
clientes_sum = clientes.groupby(['Id_cliente', 'Nombre', 'Apellido'], as_index=False)['Deuda'].sum()
print(clientes_sum)

# Se aplica un merge de Clientes y Score
base = clientes_sum.merge(scores_max, on='Id_cliente', how='left')

## Se genera una función que para un score dado, busca en la tabla de parámetros cuál es la PD que le corresponde
def obtener_pd(score):
    fila = parametros[(parametros['umbral_desde'] <= score) & (score < parametros['umbral_hasta'])]
    if not fila.empty:
        return fila['PD'].values[0]
    else:
        return None

##Se aplica la función a cada fila de la tabla para incorporar la PD de cada cliente según su Score
base['PD'] = base['Score'].apply(obtener_pd)
print(base)


   Id_cliente  Nombre Apellido  Deuda
0        1000    Pepe   Mendez  10000
1        8000    Juan    Perez  50241
2        9000  Roxana  Vazquez   8050
   Id_cliente  Nombre Apellido  Deuda  Score   PD
0        1000    Pepe   Mendez  10000     50  0.3
1        8000    Juan    Perez  50241     30  0.3
2        9000  Roxana  Vazquez   8050     10  0.5


In [ ]:
### Se generan las listas de salida solicitadas

# Clientes con peor score y mayor deuda
peor_score = base.loc[base['Score'].idxmin()]
mayor_deuda_peor = base.loc[base['Deuda'].idxmax() if base['Score'].min() == base['Score'].max() else base['Score'].idxmin()]

#  Clientes con mejor score y mayor deuda
mejor_score = base.loc[base['Score'].idxmax()]
mayor_deuda_mejor = base.loc[base['Deuda'].idxmax() if base['Score'].min() == base['Score'].max() else base['Score'].idxmax()]

### Se generan las listas de salida de modo que todos los elementos se presenten como strings
peor_lista = [
    str(peor_score['Nombre']),
    str(peor_score['Deuda']),
    str(peor_score['Score']),
    str(peor_score['PD'])
]

mejor_lista = [
    str(mejor_score['Nombre']),
    str(mejor_score['Deuda']),
    str(mejor_score['Score']),
    str(mejor_score['PD'])
]

print("Lista Peor Score y Mayor Deuda:", peor_lista)
print("Lista Mejor Score y Mayor Deuda:", mejor_lista)



Lista Peor Score y Mayor Deuda: ['Roxana', '8050', '10', '0.5']
Lista Mejor Score y Mayor Deuda: ['Pepe', '10000', '50', '0.3']


**Conclusión**
El programa permitió consolidar la información de clientes asociando la PD que le corresponde a cada cliente según su mejor score.
Además, se generaron listas que permiten identificar dentro de la base de clientes tanto a aquel de mayor riesgo como al de mejor perfil crediticio

# **8) En el año 2020 el BCRA dispuso una normativa para otorgar moratorias de pagos en los préstamos; como consecuencia por cada cuota impaga (durante el plazo de 1 año) se generó un nuevo contrato por el monto de la cuota adeuda que sería exigible al final de la vida del préstamo.**
¿Qué solución plantearía para evaluar la vida del contrato original a fin de evaluar su comportamiento de pago y recobros en las estimaciones de PD y LGD durante el periodo de moratorias y periodos posteriores?

A partir de la normativa de flexibilización de los criterios de moratoria para los deudores emitida contexto de la pandemia, se busca evaluar de manera consolidada al ciclo de vida del préstamo original, a modo de generar análisis de PD y LGD consistentes en base al riesgo real de la operación, a pesar de las cuotas reclendarizadas.

Para ello, se propone:
- Identificar al préstamo original con un ID
- Por cada uota impaga generar un nuevo ID vinculado al ID de su crédito original
- La fecha de cobro de el nuevo contrato generado por una cuota impaga, pasa a sr exigible al final del ciclo de vida del préstamo original
- La estimación de los parámetros de PD y LGD, deberá realizarse de forma consolidada sobre todo el ciclo del préstamo original, considerando tanto las cuotas pagadas normalmente como aquellas que fueron recalendarizadas y exigidas al final




In [6]:
## Ejemplo
import pandas as pd

# Se simulan los datos de los contratos, incluyendo original y los generados por moratoria
contratos = pd.DataFrame({
    'Contrato': ['A', 'A', 'A', 'A', 'B', 'C', 'D'],
    'Contrato_Madre': ['A', 'A', 'A', 'A', 'A', 'A', 'A'],
    'Cuota': [1, 2, 4, 5, 1, 1, 1],
    'Fecha_Vencimiento': ['2020-01-01', '2020-02-01', '2020-04-01', '2020-05-01',
                          '2021-01-01', '2021-01-01', '2021-01-01'],
    'Pagada': ['Sí', 'Sí', 'No', 'No', 'Sí', 'No', 'Sí'],
    'Monto': [10000, 10000, 10000, 10000, 10000, 10000, 10000]
})

# Se consolidan por contrato madre para ver el comportamiento completo del préstamo original
resumen = contratos.groupby('Contrato_Madre').agg(
    total_cuotas = ('Cuota', 'count'),
    cuotas_pagadas = ('Pagada', lambda x: (x == 'Sí').sum()),
    cuotas_impagas = ('Pagada', lambda x: (x == 'No').sum()),
    monto_total = ('Monto', 'sum')
).reset_index()

print('Detalle de contratos y cuotas:')
print(contratos)

print('Resumen consolidado por contrato original:')
print(resumen)


Detalle de contratos y cuotas:
  Contrato Contrato_Madre  Cuota Fecha_Vencimiento Pagada  Monto
0        A              A      1        2020-01-01     Sí  10000
1        A              A      2        2020-02-01     Sí  10000
2        A              A      4        2020-04-01     No  10000
3        A              A      5        2020-05-01     No  10000
4        B              A      1        2021-01-01     Sí  10000
5        C              A      1        2021-01-01     No  10000
6        D              A      1        2021-01-01     Sí  10000
Resumen consolidado por contrato original:
  Contrato_Madre  total_cuotas  cuotas_pagadas  cuotas_impagas  monto_total
0              A             7               4               3        70000
